In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,BooleanType,DoubleType
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
12,application_1678789269377_0067,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:

# function takes logs, tokens_path, output as parameter and writes the output in the given location
def generate_etherium_log_metrics(etherium_logs, tokens_df, ouput_path: str, spark):
    etherium_logs = spark.read.load(etherium_logs)
    tokens_df = spark.read.option("header","true").option("multiline","true").csv(tokens_df)

    
    etherium_logs = etherium_logs.select(['address', 'topics', 'block_timestamp', 'data'])
    etherium_logs_transfer_df=etherium_logs.filter(F.array_contains(F.col("topics"),'0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'))
    joined_tokens_logs=etherium_logs_transfer_df.join(tokens_df,['address'],'inner')
    result_df= joined_tokens_logs.withColumn('token_Value', F.expr("conv(substring(data, 3), 16, 10)")).select(['address','token_Value','block_timestamp'])
    result_time_series_df = result_df.withColumn('year',F.year(joined_tokens_logs.block_timestamp)).withColumn('month',F.month(joined_tokens_logs.block_timestamp).alias('month')).withColumn('day',F.dayofmonth(joined_tokens_logs.block_timestamp).alias('day'))

    group_columns = ['address', 'year', 'month']

    result_metrics_df = result_time_series_df.groupBy(group_columns).agg(F.count('token_Value').alias("count"), F.sum('token_Value').alias("total_token_Value"))
    result_max_occurances_df = result_metrics_df.groupBy(group_columns).agg(F.max('count').alias('count'))
    
    result_max_occurances_df = result_max_occurances_df.withColumn('count', result_max_occurances_df['count'].cast(DecimalType(18, 2)))
    result_max_occurances_df.write.mode('overwrite').options(header='true').csv(ouput_path.rstrip('/') + "/max_times/" , compression="none")

    result_max_transferred_df = result_metrics_df.groupBy(group_columns).agg(F.max('total_token_Value').alias('total_token_value'))
    result_max_transferred_df = result_max_transferred_df.withColumn('total_token_value', result_max_transferred_df.total_token_value.cast(DecimalType(18, 2)))

    result_max_transferred_df.write.mode('overwrite').options(header='true').csv(ouput_path.rstrip('/') + "/max_value/", compression="none")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
spark_session = SparkSession.builder.appName('eth_assignment').getOrCreate()

logs_path = 'gs://rwa-eth-bq-exports/logs'
tokens_path = 'path' # add path 
ouput_path = 'path' # add path where you want to write these metrics
generate_etherium_log_metrics(logs_path,tokens_path, ouput_path, spark_session)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…